In [1]:
from __future__ import print_function, division
import cv2
import torch
import numpy as np
import pickle
import math

In [2]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

In [3]:
data = pickle.load(open("toy_example/data/toyexample_1_data.p", 'rb'))
img = cv2.imread("toy_example/data/toyexample_1.png")

In [4]:
T_perturbed = np.array([[math.cos(10*math.pi/180), -math.sin(10*math.pi/180), 0, 0],
             [math.sin(10*math.pi/180), math.cos(10*math.pi/180), 0, 0],
             [0, 0, 1, 0]])

In [5]:
data['coords'] = np.around(data['2d_points']).astype(int) - 1

In [6]:
P_perturbed = np.dot(data['K'], T_perturbed)
projected_2d = np.dot(P_perturbed, np.concatenate((data['3d_points'], np.ones(len(data['3d_points']))[:, None]),-1).T)
projected_2d = (projected_2d.T/projected_2d.T[:,2,None])[:, :2]

In [7]:
coords_2d = np.around(projected_2d)
coords_2d = coords_2d.astype(int) - 1

In [8]:
data['coords'].shape , data ['3d_points'].shape

((15, 2), (15, 3))

In [9]:
img = img.astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)

0 [120 134]
1 [127  98]
2 [122 117]
3 [116  97]
4 [143 143]
5 [183 174]
6 [120 110]
7 [ 99 144]
8 [114 129]
9 [132 166]
10 [100 122]
11 [ 90 117]
12 [158 116]
13 [159 160]
14 [141 117]


In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.imwrite('perturbed_points.png', img)

True

### run BA

In [11]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('double')

In [13]:
img_torch = torch.from_numpy(img)[None,...]
grad_x, grad_y = sobel_filter(img_torch)

In [14]:
cv2.imshow('image', grad_x.numpy().reshape(img.shape).astype('uint8'))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
cv2.imwrite("grad_x.png", grad_x.numpy().reshape(img.shape).astype('uint8'))
cv2.imwrite("grad_y.png", grad_y.numpy().reshape(img.shape).astype('uint8'))

True

In [16]:
pts3D = torch.from_numpy(data['3d_points'][:,:3])
ref2d = torch.from_numpy(np.flip(data['coords'], axis = 1).copy())
feature_ref = torch.cat([img_torch[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0], ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = img_torch.type(torch.DoubleTensor)
R_init, t_init = torch.from_numpy(T_perturbed[:, :3]), torch.from_numpy(T_perturbed[:, 3])
feature_grad_x = grad_x
feature_grad_y = grad_y
K = torch.from_numpy(data['K'])

In [426]:
from __future__ import print_function, division

from featureBA.src.utils import (from_homogeneous, to_homogeneous,
                                 batched_eye_like, skew_symmetric, so3exp_map)

from featureBA.src.utils import squared_loss, scaled_loss
import torch
from torch import nn
import numpy as np

def optimizer_step(g, H, lambda_=0):
    """One optimization step with Gauss-Newton or Levenberg-Marquardt.
    Args:
        g: batched gradient tensor of size (..., N).
        H: batched hessian tensor of size (..., N, N).
        lambda_: damping factor for LM (use GN if lambda_=0).
    """
    if lambda_:  # LM instead of GN
        D = (H.diagonal(dim1=-2, dim2=-1) + 1e-9).diag_embed()
        H = H + D*lambda_
    try:
        P = torch.inverse(H)
    except RuntimeError as e:
        logging.warning(f'Determinant: {torch.det(H)}')
        raise e
    delta = -(P @ g[..., None])[..., 0]
    return delta

def indexing_(feature_map, points):
    '''
    Function gives x and y gradients for 3D points in camera frame.

    inputs: (All pytorch tensors)
    @feature_map : x gradient of the feature map (CxHxW)
    @points : pixel coordinates of points (Nx2)

    outputs: 
    features : features for the points (NxC)
    '''

    features = torch.cat([feature_map[:, i, j].unsqueeze(0) for i, j in zip(points[:,0], points[:,1])])

    return features

In [427]:
n_iters = 100
loss_fn = squared_loss
lambda_ = 0.01

R = R_init
t = t_init

In [432]:
p_3d_1 = (torch.mm(R, pts3D.T).T + t)
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
print((error))

tensor([[ -63.],
        [ -34.],
        [ -14.],
        [ -14.],
        [ -28.],
        [  -7.],
        [ -28.],
        [  -7.],
        [ -34.],
        [ -52.],
        [-130.],
        [ -63.],
        [ -28.]], dtype=torch.float64)


In [434]:
i = 0
verbose = 1
p_3d_1 = torch.mm(R, pts3D.T).T + t
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
print(error)
cost = 0.5*(error**2).sum(-1)

if i == 0:
    prev_cost = cost.mean(-1)
if verbose:
    print('Iter ', i, cost.mean().item())

tensor([[ -63.],
        [ -34.],
        [ -14.],
        [ -14.],
        [ -28.],
        [  -7.],
        [ -28.],
        [  -7.],
        [ -34.],
        [ -52.],
        [-130.],
        [ -63.],
        [ -28.]], dtype=torch.float64)
Iter  0 1257.5384615384614


In [441]:
J_p_T = torch.cat([
    batched_eye_like(p_3d_1, 3), -skew_symmetric(p_3d_1)], -1)
print("J_p_T is ", J_p_T.shape)
shape = p_3d_1.shape[:-1]
o, z = p_3d_1.new_ones(shape), p_3d_1.new_zeros(shape)
J_e_p = torch.stack([
    o, z, -p_3d_1[..., 0] / p_3d_1[..., 2],
    z, o, -p_3d_1[..., 1] / p_3d_1[..., 2],
], dim=-1).reshape(shape+(2, 3)) / p_3d_1[..., 2, None, None]

print("J_e_p is ", J_e_p.shape)

grad_x_points = indexing_(feature_grad_x, torch.flip(p_proj_1,(1,)))
grad_y_points = indexing_(feature_grad_y, torch.flip(p_proj_1,(1,)))

J_p_F = torch.cat((grad_x_points[..., None], grad_y_points[...,None]), -1)

print("J_p_F is ", J_p_F.shape)
J_e_T = J_p_F @ J_e_p @ J_p_T

Grad = torch.einsum('...ijk,...ij->...ik', J_e_T, error)
Grad = Grad.sum(-2)  # Grad was ... x N x 6

J_p_T is  torch.Size([13, 3, 6])
J_e_p is  torch.Size([13, 2, 3])
J_p_F is  torch.Size([13, 1, 2])


In [ ]:
J = J_e_T
Hess = torch.einsum('...ijk,...ijl->...ikl', J, J)
# Hess = weights[..., None, None] * Hess
Hess = Hess.sum(-3)  # Hess was ... x N x 6 x 6

delta = optimizer_step(Grad, Hess, lambda_)

dt, dw = delta[..., :3], delta[..., 3:6]
dr = so3exp_map(dw)
print("dr is : ", dw)
print("dt is : ", dt)
R_new = dr @ R
t_new = dr @ t + dt

new_proj_1 = from_homogeneous(pts3D @R_new.T + t_new).type(torch.IntTensor)
new_error = indexing_(feature_map_query, new_proj_1) - feature_ref
new_cost = (new_error**2).sum(-1)
# new_cost = scaled_loss(new_cost, self.loss_fn, scale[..., None])[0]
# new_cost = (confidence*new_cost).mean(-1)
new_cost = new_cost.mean(-1)

lambda_ = np.clip(lambda_ * (10 if new_cost > prev_cost else 1/10),
                  1e-6, 1e2)
# if new_cost > prev_cost:  # cost increased
#     print("cost increased, continue with next iteration")
#     continue
prev_cost = new_cost
R, t = R_new, t_new

In [442]:
i = 0
verbose = 1
p_3d_1 = torch.mm(R, pts3D.T).T + t
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
print(error)
cost = 0.5*(error**2).sum(-1)

if i == 0:
    prev_cost = cost.mean(-1)
if verbose:
    print('Iter ', i, cost.mean().item())

J_p_T = torch.cat([
    batched_eye_like(p_3d_1, 3), -skew_symmetric(p_3d_1)], -1)
# print("J_p_T is ", J_p_T)
shape = p_3d_1.shape[:-1]
o, z = p_3d_1.new_ones(shape), p_3d_1.new_zeros(shape)
J_e_p = torch.stack([
    o, z, -p_3d_1[..., 0] / p_3d_1[..., 2],
    z, o, -p_3d_1[..., 1] / p_3d_1[..., 2],
], dim=-1).reshape(shape+(2, 3)) / p_3d_1[..., 2, None, None]

# print("J_e_p is ", J_e_p)

grad_x_points = indexing_(feature_grad_x, torch.flip(p_proj_1,(1,)))
grad_y_points = indexing_(feature_grad_y, torch.flip(p_proj_1,(1,)))

J_p_F = torch.cat((grad_x_points[..., None], grad_y_points[...,None]), -1)

# print("J_p_F is ", J_p_F)
J_e_T = J_p_F @ J_e_p @ J_p_T

Grad = torch.einsum('...ijk,...ij->...ik', J_e_T, error)
# Grad = weights[..., None] * Grad
Grad = Grad.sum(-2)  # Grad was ... x N x 6

J = J_e_T
Hess = torch.einsum('...ijk,...ijl->...ikl', J, J)
# Hess = weights[..., None, None] * Hess
Hess = Hess.sum(-3)  # Hess was ... x N x 6 x 6

delta = optimizer_step(Grad, Hess, lambda_)
# if torch.isnan(delta).any():
#     logging.warning('NaN detected, exit')
#     break
dt, dw = delta[..., :3], delta[..., 3:6]
dr = so3exp_map(dw)
print("dr is : ", dw)
print("dt is : ", dt)
R_new = dr @ R
t_new = dr @ t + dt

new_proj_1 = from_homogeneous(torch.matmul(K, (pts3D @R_new.T + t_new).T).T).type(torch.IntTensor)
new_error = indexing_(feature_map_query, new_proj_1) - feature_ref
new_cost = (new_error**2).sum(-1)
# new_cost = scaled_loss(new_cost, self.loss_fn, scale[..., None])[0]
# new_cost = (confidence*new_cost).mean(-1)
new_cost = new_cost.mean(-1)

lambda_ = np.clip(lambda_ * (10 if new_cost > prev_cost else 1/10),
                  1e-6, 1e2)
# if new_cost > prev_cost:  # cost increased
#     print("cost increased, continue with next iteration")
#     continue
prev_cost = new_cost
R, t = R_new, t_new

tensor([[ -63.],
        [ -34.],
        [ -14.],
        [ -14.],
        [ -28.],
        [  -7.],
        [ -28.],
        [  -7.],
        [ -34.],
        [ -52.],
        [-130.],
        [ -63.],
        [ -28.]], dtype=torch.float64)
Iter  0 1257.5384615384614
J_p_T is  tensor([[[  1.0000,   0.0000,   0.0000,  -0.0000,   1.9287,   5.9906],
         [  0.0000,   1.0000,   0.0000,  -1.9287,  -0.0000, -12.0069],
         [  0.0000,   0.0000,   1.0000,  -5.9906,  12.0069,  -0.0000]],

        [[  1.0000,   0.0000,   0.0000,  -0.0000,   1.7825,  -6.0832],
         [  0.0000,   1.0000,   0.0000,  -1.7825,  -0.0000,  -9.3175],
         [  0.0000,   0.0000,   1.0000,   6.0832,   9.3175,  -0.0000]],

        [[  1.0000,   0.0000,   0.0000,  -0.0000,   2.6854,   8.0988],
         [  0.0000,   1.0000,   0.0000,  -2.6854,  -0.0000,  -4.7485],
         [  0.0000,   0.0000,   1.0000,  -8.0988,   4.7485,  -0.0000]],

        [[  1.0000,   0.0000,   0.0000,  -0.0000,   1.5835,   2.5565],
    

In [17]:
model = sparse3DBA(n_iters = 100, lambda_ = 0)

In [18]:
R, t = model(pts3D, feature_ref, feature_map_query, feature_grad_x, feature_grad_y, K, R_init, t_init)

prev cost is  tensor(4870.1333, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 7.8887e-05, -7.1807e-05,  1.4058e-04], dtype=torch.float64)
dt is :  tensor([ 5.5855e-04,  7.1423e-05, -2.0175e-05], dtype=torch.float64)
new cost is  tensor(5260., dtype=torch.float64)
prev cost is  tensor(5260., dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 7.8809e-05, -7.1985e-05,  1.4047e-04], dtype=torch.float64)
dt is :  tensor([ 5.5924e-04,  6.8002e-05, -2.0155e-05], dtype=torch.float64)
new cost is  tensor(5418.5000, dtype=torch.float64)
prev cost is  tensor(5418.5000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 7.8730e-05, -7.2159e-05,  1.4035e-04], dtype=torch.float64)
dt is :  tensor([ 5.5987e-04,  6.4613e-05, -2.0063e-05], d

new cost is  tensor(5732.2333, dtype=torch.float64)
prev cost is  tensor(5732.2333, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 4.6434e-05, -8.0797e-05,  9.0722e-05], dtype=torch.float64)
dt is :  tensor([ 0.0009, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(5676.7667, dtype=torch.float64)
prev cost is  tensor(5676.7667, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 4.5788e-05, -8.0173e-05,  8.9642e-05], dtype=torch.float64)
dt is :  tensor([ 0.0009, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(5676.7667, dtype=torch.float64)
prev cost is  tensor(5676.7667, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 4.5697e-05, -8.0108e-05,  8.9641e-05], dtype=torch.float64)
dt is :  tensor([ 0.00

dr is :  tensor([ 2.4662e-05, -6.1066e-05,  8.7624e-05], dtype=torch.float64)
dt is :  tensor([ 0.0006, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(6120.1000, dtype=torch.float64)
prev cost is  tensor(6120.1000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 2.4609e-05, -6.0991e-05,  8.7654e-05], dtype=torch.float64)
dt is :  tensor([ 0.0006, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(6120.1000, dtype=torch.float64)
prev cost is  tensor(6120.1000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 2.0756e-05, -6.5437e-05,  7.8327e-05], dtype=torch.float64)
dt is :  tensor([ 0.0004, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(6120.1000, dtype=torch.float64)
prev cost is  tensor(6120.1000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([

J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 2.4944e-05, -6.4265e-05,  6.6607e-05], dtype=torch.float64)
dt is :  tensor([ 5.1636e-04, -3.9282e-05,  1.6593e-04], dtype=torch.float64)
new cost is  tensor(6504.7000, dtype=torch.float64)
prev cost is  tensor(6504.7000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 2.4895e-05, -6.4192e-05,  6.6669e-05], dtype=torch.float64)
dt is :  tensor([ 5.1594e-04, -3.9849e-05,  1.6523e-04], dtype=torch.float64)
new cost is  tensor(6504.7000, dtype=torch.float64)
prev cost is  tensor(6504.7000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 2.4845e-05, -6.4119e-05,  6.6731e-05], dtype=torch.float64)
dt is :  tensor([ 5.1552e-04, -4.0417e-05,  1.6453e-04], dtype=torch.float64)
new cost is  tensor(6504.7000, dtype=torch.float64)
prev cost is  tensor(6504.7000, dtype=t

new cost is  tensor(6729.5000, dtype=torch.float64)
prev cost is  tensor(6729.5000, dtype=torch.float64)
J_p_T is  torch.Size([15, 3, 6])
J_e_p is  torch.Size([15, 2, 3])
J_p_F is  torch.Size([15, 1, 2])
dr is :  tensor([ 1.3844e-05, -4.3357e-05,  7.9642e-05], dtype=torch.float64)
dt is :  tensor([ 0.0004, -0.0001,  0.0001], dtype=torch.float64)
new cost is  tensor(6729.5000, dtype=torch.float64)


In [19]:
R

tensor([[ 0.9832, -0.1825, -0.0068],
        [ 0.1825,  0.9832, -0.0035],
        [ 0.0073,  0.0022,  1.0000]], dtype=torch.float64)

In [23]:
t_init, t

(tensor([0., 0., 0.], dtype=torch.float64),
 tensor([ 0.0588, -0.0126,  0.0119], dtype=torch.float64))

In [348]:
a[0] @ b[0]

array([[1.13206942, 1.11894092, 0.57274033, 1.24841041, 0.79335565,
        0.46561771],
       [1.2855546 , 1.19792658, 1.26153542, 1.5414339 , 1.17330447,
        1.04735982]])

In [349]:
a @ b

array([[[1.13206942, 1.11894092, 0.57274033, 1.24841041, 0.79335565,
         0.46561771],
        [1.2855546 , 1.19792658, 1.26153542, 1.5414339 , 1.17330447,
         1.04735982]]])